## 라이브러리 불러오기

In [2]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

## 데이터 불러오기

In [3]:
# 21년 자료 train 데이터, 22년 자료 val 데이터
train = pd.read_csv('df_21.csv', index_col='time', parse_dates=True)
val = pd.read_csv('df_22.csv', index_col='time', parse_dates=True)

In [4]:
train.head()

,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),...,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),PM10_t+1,PM25_t+1,year,month,day,hour
time,,,,,,,,,,,,,,,,,,,,,
2021-10-01 01:00:00,0.003,0.6,0.002,0.039,31.0,18.0,18.7,0.1,1.0,20.0,...,22.1,22.0,22.5,22.9,27.0,16.0,2021,10,1,1
2021-10-01 02:00:00,0.003,0.6,0.002,0.035,27.0,16.0,18.3,0.1,0.3,0.0,...,21.8,21.8,22.4,22.8,28.0,18.0,2021,10,1,2
2021-10-01 03:00:00,0.003,0.6,0.002,0.033,28.0,18.0,17.7,0.1,2.0,20.0,...,21.6,21.6,22.2,22.8,26.0,16.0,2021,10,1,3
2021-10-01 04:00:00,0.003,0.6,0.002,0.030,26.0,16.0,17.4,0.1,1.0,50.0,...,21.3,21.4,22.0,22.7,26.0,16.0,2021,10,1,4
2021-10-01 05:00:00,0.003,0.5,0.003,0.026,26.0,16.0,17.2,0.1,1.3,270.0,...,21.1,21.2,21.9,22.6,22.0,14.0,2021,10,1,5


In [5]:
val.head()

,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),...,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),PM10_t+1,PM25_t+1,year,month,day,hour
time,,,,,,,,,,,,,,,,,,,,,
2022-01-02 01:00:00,0.004,0.8,0.002,0.052,38.0,24.0,-2.9,0.00,2.1,50,...,-0.4,-0.7,-0.3,0.8,34.0,23.0,2022,1,2,1
2022-01-02 02:00:00,0.004,0.8,0.002,0.052,34.0,23.0,-2.6,0.00,1.9,50,...,-0.4,-0.7,-0.3,0.8,35.0,26.0,2022,1,2,2
2022-01-02 03:00:00,0.004,0.8,0.002,0.052,35.0,26.0,-2.1,0.00,2.5,50,...,-0.4,-0.7,-0.3,0.8,33.0,24.0,2022,1,2,3
2022-01-02 04:00:00,0.004,0.6,0.002,0.046,33.0,24.0,-1.9,0.15,2.0,50,...,-0.3,-0.7,-0.3,0.8,33.0,25.0,2022,1,2,4
2022-01-02 05:00:00,0.003,0.5,0.005,0.039,33.0,25.0,-1.9,0.15,2.3,50,...,-0.3,-0.7,-0.3,0.8,32.0,23.0,2022,1,2,5


In [6]:
# y값은 PM10_t+1, PM25_t+1
# x값은 y값을 제외한 나머지
train_x = train.drop(columns=["PM10_t+1", "PM25_t+1"])
train_y = train[["PM10_t+1", "PM25_t+1"]]

val_x = val.drop(columns=["PM10_t+1", "PM25_t+1"])
val_y = val[["PM10_t+1", "PM25_t+1"]]

## 학습 및 평가

#### H2O 패키지 준비

In [7]:
# H2O 패키지 초기화
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/hwl/anaconda3/envs/Air_Quality/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpts0ueok9
  JVM stdout: /tmp/tmpts0ueok9/h2o_hwl_started_from_python.out
  JVM stderr: /tmp/tmpts0ueok9/h2o_hwl_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_hwl_uauur5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.734 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [8]:
# h2o 자체 데이터 프레임으로 변환
train_hf = h2o.H2OFrame(train)
val_hf = h2o.H2OFrame(val)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
x = train_x.columns.tolist()
y = ["PM10_t+1", "PM25_t+1"]

#### H2O Automl 학습

In [10]:
# PM10_t+1 예측모델 학습
aml_PM10 = H2OAutoML(max_models=20, seed=1)
aml_PM10.train(x=x, y="PM10_t+1", training_frame=train_hf)

AutoML progress: |
12:39:30.748: _train param, Dropping bad and constant columns: [year]

██
12:39:40.209: _train param, Dropping bad and constant columns: [year]
12:39:40.902: _train param, Dropping bad and constant columns: [year]

███
12:39:54.836: _train param, Dropping bad and constant columns: [year]


12:39:58.861: _train param, Dropping bad and constant columns: [year]

██
12:40:07.996: _train param, Dropping bad and constant columns: [year]

█
12:40:10.682: _train param, Dropping bad and constant columns: [year]


12:40:13.663: _train param, Dropping bad and constant columns: [year]

█
12:40:16.898: _train param, Dropping bad and constant columns: [year]

█
12:40:18.972: _train param, Dropping bad and constant columns: [year]

████
12:40:30.848: _train param, Dropping bad and constant columns: [year]


12:40:33.123: _train param, Dropping bad and constant columns: [year]

████████████████████████████████████████████
12:47:25.308: _train param, Dropping unused columns: [year]
1

key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/20
# GBM base models (used / total),3/7
# XGBoost base models (used / total),4/6
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
# DeepLearning base models (used / total),1/4
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [11]:
# PM25_t+1 예측모델 학습
aml_PM25 = H2OAutoML(max_models=20, seed=1)
aml_PM25.train(x=x, y="PM25_t+1", training_frame=train_hf)

AutoML progress: |


12:47:28.245: _train param, Dropping bad and constant columns: [year]

█
12:47:31.809: _train param, Dropping bad and constant columns: [year]
12:47:31.966: _train param, Dropping bad and constant columns: [year]

███
12:47:37.703: _train param, Dropping bad and constant columns: [year]

██
12:47:39.721: _train param, Dropping bad and constant columns: [year]

██
12:47:47.826: _train param, Dropping bad and constant columns: [year]

█
12:47:50.29: _train param, Dropping bad and constant columns: [year]


12:47:52.621: _train param, Dropping bad and constant columns: [year]

██
12:47:55.835: _train param, Dropping bad and constant columns: [year]


12:47:57.907: _train param, Dropping bad and constant columns: [year]

████
12:48:08.241: _train param, Dropping bad and constant columns: [year]


12:48:10.129: _train param, Dropping bad and constant columns: [year]

███████████████████████████████████████████
12:54:42.181: _train param, Dropping unused columns: [year]
12:54:42.621: _train

key,value
Stacking strategy,cross_validation
Number of base models (used / total),14/20
# GBM base models (used / total),6/7
# XGBoost base models (used / total),6/6
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
# DeepLearning base models (used / total),2/4
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


##### 리더보드 확인

In [12]:
lb_PM10 = aml_PM10.leaderboard
print(lb_PM10.head(rows=lb_PM10.nrows))

model_id                                                     rmse       mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20230925_123930      7.91627   62.6673  3.67703    0.158401                   62.6673
StackedEnsemble_BestOfFamily_1_AutoML_1_20230925_123930   7.95906   63.3466  3.73543    0.161418                   63.3466
GBM_grid_1_AutoML_1_20230925_123930_model_2               8.11927   65.9226  3.95792    0.171233                   65.9226
XGBoost_grid_1_AutoML_1_20230925_123930_model_3           8.20001   67.2402  3.88335    0.169127                   67.2402
XRT_1_AutoML_1_20230925_123930                            8.29882   68.8704  3.89689    0.16543                    68.8704
GBM_5_AutoML_1_20230925_123930                            8.38241   70.2648  3.88584    0.165051                   70.2648
DRF_1_AutoML_1_20230925_123930                            8.59754   73.9176  3.9205     0.166199                   73.9176
GBM_4_AutoML_1_2

In [13]:
lb_PM25 = aml_PM25.leaderboard
print(lb_PM25.head(rows=lb_PM25.nrows))

model_id                                                    rmse       mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_2_20230925_124728     3.14047   9.86255  2.13138    0.18109                    9.86255
StackedEnsemble_BestOfFamily_1_AutoML_2_20230925_124728  3.24807  10.55     2.1882     0.185273                  10.55
GBM_3_AutoML_2_20230925_124728                           3.26948  10.6895   2.21766    0.186202                  10.6895
GBM_2_AutoML_2_20230925_124728                           3.29052  10.8275   2.2202     0.186755                  10.8275
GBM_4_AutoML_2_20230925_124728                           3.31098  10.9626   2.23198    0.186742                  10.9626
GBM_5_AutoML_2_20230925_124728                           3.32371  11.0471   2.24098    0.188774                  11.0471
GBM_grid_1_AutoML_2_20230925_124728_model_1              3.33325  11.1106   2.30294    0.193648                  11.1106
GBM_grid_1_AutoML_2_20230925_124

#### H2O Automl 예측

In [14]:
best_model_PM10 = aml_PM10.leader
preds_val_y_PM10 = best_model_PM10.predict(val_hf)

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


In [15]:
best_model_PM25 = aml_PM25.leader
preds_val_y_PM25 = best_model_PM25.predict(val_hf)

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


In [16]:
# metric 확인
perf_PM10 = best_model_PM10.model_performance(val_hf)
perf_PM25 = best_model_PM25.model_performance(val_hf)
print(perf_PM10)
print(perf_PM25)

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 35.96472414917394
RMSE: 5.997059625280871
MAE: 3.954728720280761
RMSLE: 0.18434092334101426
Mean Residual Deviance: 35.96472414917394
R^2: 0.9352192630557019
Null degrees of freedom: 2094
Residual degrees of freedom: 2085
Null deviance: 1169470.2067449545
Residual deviance: 75346.0970925194
AIC: 13472.7707652121
ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 26.01375177351363
RMSE: 5.100367807669721
MAE: 3.4776767129776363
RMSLE: 0.20808374380876943
Mean Residual Deviance: 26.01375177351363
R^2: 0.9245209587810818
Null degrees of freedom: 2094
Residual degrees of freedom: 2080
Null deviance: 818057.8082864622
Residual deviance: 54498.809965511056
AIC: 12804.1724839439


In [17]:
# 모델 저장
model_path_PM10 = h2o.save_model(model=best_model_PM10, path="./PM10_model", force=True)
model_path_PM25 = h2o.save_model(model=best_model_PM25, path="./PM25_model", force=True)